In [ ]:
!pip install -q tensorflow==2.13.0
!apt-get -q update && apt-get install -q -y python3-opencv

In [ ]:
class color:
    BOLD = '\033[1m'
    BOLD_COLOR = '\033[1m' + '\033[34m'
    END = '\033[0m'

print(color.BOLD_COLOR + 'Importing libraries.....' + color.END)    

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore',category = ConvergenceWarning)
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)

import cv2
import itertools
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report, f1_score , confusion_matrix

# Tensorflow Libraries
import tensorflow as tf
import keras
from keras.layers import Dense,Dropout,BatchNormalization
from keras.optimizers import Adam
from keras import layers,models,Model
from tensorflow.keras.layers.experimental import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.applications import ResNet50V2
from keras.utils import plot_model


# print("\nNum GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(color.BOLD + '\nTensorflow Version: {}'.format(tf.__version__))
# AUTO = tf.data.experimental.AUTOTUNE
print('\nKeras Version: {}'.format(keras.__version__) + color.END)

print(color.BOLD_COLOR + '\nDone!')

In [ ]:
class CFG:
    device = 'TPU'

In [ ]:
def get_device():

    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        print(f'> Running on TPU', tpu.master(),end = ' | ')
        print('Num of TPUs: ', strategy.num_replicas_in_sync)
        divice = CFG.device
    except:
        gpus = tf.config.list_logical_devices('GPU')
        ngpu = len(gpus)

        if ngpu:
            strategy = tf.distribute.MirroredStrategy(gpus)
            print('> Running on GPU', end = ' | ')
            print('Num of GPUs: ', ngpu)
            device = 'GPU'
        else:
            print('> Running on GPU')
            strategy = tf.distribute.get_strategy()
            device = 'CPU'

    return strategy,device

In [ ]:
strategy, CFG.device = get_device()
CFG.replicas = strategy.num_replicas_in_sync

In [ ]:
path = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"

data = {"imgpath": [] , "labels": [] }

category = os.listdir(path)
for folder in category:
    folderpath = os.path.join(path , folder)
    filelist = os.listdir(folderpath)
    for file in filelist:
        fpath = os.path.join(folderpath, file)
        data["imgpath"].append(fpath)
        data["labels"].append(folder)

df = pd.DataFrame(data) 

# Convert labels to numbers
# lb = LabelEncoder()
# df['encoded_labels'] = lb.fit_transform(df['labels'])

In [ ]:
train_df, Temp_df = train_test_split(df,  train_size= 0.70 , shuffle=True, random_state=124)
valid_df , test_df = train_test_split(Temp_df ,  train_size= 0.70 , shuffle=True, random_state=124)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(color.BOLD_COLOR + "----------Train-------------" + color.END)
print(color.BOLD + train_df[["imgpath", "labels"]].head(5))
print(color.BOLD + 'Shape Train: {}'.format(train_df.shape))
print(color.BOLD_COLOR + "--------Validation----------" + color.END)
print(color.BOLD  + valid_df[["imgpath", "labels"]].head(5))
print(color.BOLD + 'Shape Validation: {}'.format(valid_df.shape))
print(color.BOLD_COLOR + "----------Test--------------" + color.END)
print(color.BOLD + test_df[["imgpath", "labels"]].head(5))
print(color.BOLD + 'Shape Test: {}'.format(test_df.shape))

In [ ]:
plt.figure(figsize=(15,12))
for i, row in test_df.sample(n=16).reset_index().iterrows():
    plt.subplot(4,4,i+1)
    image_path = row['imgpath']
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(row["labels"])
    plt.axis('off')
plt.show()

# ResNet50V2


In [ ]:
# train_dir = '/kaggle/input/animal-image-dataset-90-different-animals/animals/animals'
RESNE_VALIDATION_SIZE = 64
RESNET_BATCH_SIZE = 24
RESNET_IMAGE_SIZE =(224,224)
RESNET_CLASS_MODE = 'categorical'

In [ ]:
%%time
print(color.BOLD)

resnet_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    fill_mode = 'nearest',
    horizontal_flip = True,
    vertical_flip = True,
    #validation_split = 0.3
)

train_resnest = resnet_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col='imgpath',
    y_col='labels',
    target_size = RESNET_IMAGE_SIZE,
    class_mode = RESNET_CLASS_MODE,
    batch_size = RESNET_BATCH_SIZE,
    shuffle = True,
    color_mode='rgb'
)

In [ ]:
%%time
print(color.BOLD)

resnet_validation_generator = ImageDataGenerator(
    rescale = 1./255
)

validation_resnet = resnet_validation_generator.flow_from_dataframe(
    dataframe=valid_df,
    x_col='imgpath',
    y_col = 'labels',
    target_size=RESNET_IMAGE_SIZE,
    class_mode= RESNET_CLASS_MODE,
    batch_size=RESNE_VALIDATION_SIZE,
    color_mode = 'rgb'
)


test_resnet = resnet_validation_generator.flow_from_dataframe(
    dataframe= test_df,
    x_col = 'imgpath',
    y_col = 'labels',
    target_size=RESNET_IMAGE_SIZE,
    class_mode = RESNET_CLASS_MODE,
    batch_size=RESNE_VALIDATION_SIZE,
    color_mode='rgb',
   #subset = 'validation'
)

In [ ]:
print(color.BOLD)
print(train_resnest.class_indices)
print(validation_resnet.class_indices)
print(test_resnet.class_indices)
print(color.END)

In [ ]:
print(color.BOLD)
conv_base = ResNet50V2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

In [ ]:
conv_base.trainable = False

In [ ]:
num_classes = len(set(train_resnest.classes))

model_resnet50 = keras.models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(548,use_bias=False,activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.45),
    layers.Dense(num_classes,activation = 'softmax')
])

print(color.BOLD_COLOR + '\nVanilla Model layers and output shapes with params...\n'+color.END)

print(color.BOLD)
model_resnet50.summary()

In [ ]:
print(color.BOLD_COLOR + '\nSetting a decay learning rate for learning rate schedule\n' + color.END)
epoch = 50
learning_rate = 3e-5
lr_start = 0.0005
lr_min = 0.00001
lr_max = 0.0005
lr_rampup_epochs = 1
lr_sustain_epochs = 1
lr_exp_decay = .8

def lrfn(epoch):
    if epoch < lr_rampup_epochs:
        lr = (lr_max - lr_start)/lr_rampup_epochs * epoch + lr_start
    elif epoch < lr_rampup_epochs + lr_sustain_epochs:
        lr = lr_max
    else:
        lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
    return lr

print(color.BOLD_COLOR + 'Done!' + color.END)

In [ ]:
colors = ['#36609A', '#FFCE30', '#194350']
epochs = 20
epochs_range = [i for i in range(50 if epochs < 50 else epochs)]
learning_rate = [lrfn(x) for x in epochs_range]

fig,ax = plt.subplots(figsize = (10,5))
fig.patch.set_facecolor('#f5f6f6')
ax.set_facecolor('#f5f6f6')

for loc in ['right','top']:
    ax.spines[loc].set_visible(False)
    
ax.plot(epochs_range,learning_rate,linewidth = 4,color = colors[0])
plt.xlabel('Range of epochs',
           {'fontfamily':'serif','size':14,'color':'black','weight':'bold'})
plt.ylabel('Learning rate in 10^-5',
           {'fontfamily':'serif','size':14,'color':'black','weight':'bold'})

plt.gcf().text(0,1.06,'Learning Rate Schedule',
               {'fontfamily':'serif','size': 24,'color':'black','weight':'bold'})

plt.gcf().text(0.75,0,'Make by trinhminh/Kaggle',
               {'fontfamily':'serif','size':10,'color':'black'})

plt.gcf().show()

In [ ]:
print(color.BOLD_COLOR + '\nSetting early stopping facor and learning rate schedule\n' + color.END)

earlystop = EarlyStopping(patience=5)
lr_callback = keras.callbacks.LearningRateScheduler(lrfn,verbose=True)

callbacks = [earlystop,lr_callback]
print(color.BOLD_COLOR + 'Done!')

In [ ]:
# plot_model(model_resnet50,show_shapes=True,expand_nested=True,dpi = 80)

In [ ]:
model_resnet50.compile(
    optimizer=Adam(0.0005),
    loss = keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
print(color.BOLD)

history_resnet50 = model_resnet50.fit(
    train_resnest,
    validation_data=test_resnet,
    epochs = 15,
    callbacks=[
        EarlyStopping(monitor = "val_loss",patience = 3,restore_best_weights = True), 
        # if val loss decreases for 10 epochs in a row, stop training,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min') 
    ]
)

In [ ]:
model_resnet50.compile(
    optimizer=Adam(0.0001),
    loss = keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
print(color.BOLD)

history_resnet50 = model_resnet50.fit(
    train_resnest,
    validation_data=test_resnet,
    epochs = 10,#15#15
    callbacks=[
        EarlyStopping(monitor = "val_loss",patience = 4,restore_best_weights = True), 
        # if val loss decreases for 10 epochs in a row, stop training,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min') 
    ]
)

In [ ]:
print(color.BOLD)
model_resnet50.evaluate(validation_resnet)

In [ ]:
import seaborn as sns

fig,ax = plt.subplots(2,1,figsize = (8,8),dpi=100)
fig.patch.set_facecolor('#f5f6f6')

axes = ax.ravel()

for ax in axes:
    ax.set_facecolor('#f5f6f6')
    for loc in ['right','top']:
        ax.spines[loc].set_visible(False)
        
        
hist2 = history_resnet50.history
Epochs = range(len(hist2['loss']))

sns.lineplot(x = Epochs,y=hist2['val_loss'], ax = axes[0],linewidth = 4, color = colors[0])
sns.lineplot(x = Epochs,y=hist2['loss'],ax = axes[0],linewidth = 4,color = colors[1])

axes[0].text(Epochs[-1]+0.25,hist2['val_loss'][-1],'Validation Loss',
            {'fontfamily': 'serif','size':12,'weight':'bold','color':colors[0]})
axes[0].text(Epochs[-1]+0.25,hist2['loss'][-1],'Training Loss',
             {'fontfamily':'serif','size':12,'weight':'bold','color':colors[1]})


sns.lineplot(x = Epochs,y=hist2['val_accuracy'],ax = axes[1],linewidth=4,color = colors[0])
sns.lineplot(x = Epochs,y=hist2['accuracy'],ax = axes[1],linewidth=4,color = colors[1])

axes[1].text(Epochs[-1]+0.25,hist2['val_accuracy'][-1],'Validation Accuracy',
             {'fontfamily':'serif','size':12,'weight':'bold','color':colors[0]})
axes[1].text(Epochs[-1]+0.25,hist2['accuracy'][-1],'Training Accuracy',
             {'fontfamily':'serif','size':12,'weight':'bold','color':colors[1]})



fig.text(0,1.06, 'ResNet Loss and Accuracy',
         {'fontfamily':'serif','size':18,'weight':'bold'})
fig.text(0,1.01,'As expected transfer learning did a good jod with a lomost negligable'
        + '\ndifference between validation and training loss and accuracies.',
         {'fontfamily':'serif','size':12})
fig.text(0.75,-0.05,'Made by trinhminnh/Kaggle',
         {'fontfamily':'serif','size':10,'color':'black'})

plt.tight_layout(h_pad = 5)

fig.show()

In [ ]:
y_true = validation_resnet.classes
y_pre = np.argmax(model_resnet50.predict(validation_resnet),axis=1)
f1 = f1_score(y_true,y_pre,average='macro')
print('F1 Score: {}'.format(f1))
print(classification_report(y_true,y_pre,target_names = validation_resnet.class_indices.keys()))

In [ ]:
classes = dict(zip(validation_resnet.class_indices.values(),validation_resnet.class_indices.keys()))
Predictions = pd.DataFrame({'Image Index': list(range(len(validation_resnet.labels))),
                           'Test Labels': validation_resnet.labels,
                           'Test Classes': [classes[i] for i in validation_resnet.labels],
                            'Prediction Labels': y_pre,
                            'Prediction Classes': [classes[i] for i in y_pre],
                            'Path': validation_resnet.filenames,
                            'Prediction Probability': [x for x in np.asarray(tf.reduce_max(model_resnet50.predict(validation_resnet),axis = 1))]
                           })

Predictions.head(20)

In [ ]:
plt.figure(figsize=(20,20))
for i,row in Predictions[Predictions['Test Labels'] == Predictions['Prediction Labels']].sort_values('Prediction Probability').tail(20).reset_index().iterrows():
    plt.subplot(5,4,i+1)
    image_path = row['Path']
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(f'TRUE: {row["Test Classes"]} | PRED: {row["Prediction Classes"]}', fontsize=8)
    plt.axis('off')
    
plt.show()

In [ ]:
categori_classname = ['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle',
                      'bison', 'boar', 'butterfly', 'cat', 'caterpillar',
                      'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab',
                      'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly',
                      'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox',
                      'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper',
                      'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill',
                      'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo',
                      'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster',
                      'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan',
                      'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes',
                      'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat',
                      'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark',
                      'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish',
                      'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf',
                      'wombat', 'woodpecker', 'zebra']

In [ ]:
test_img = cv2.imread('/kaggle/input/animal-image-dataset-90-different-animals/animals/animals/dog/8c6a936c9a.jpg')

test_img_resized = cv2.resize(test_img, (224, 224))

test_input = np.array(test_img_resized)

test_input = np.expand_dims(test_input, axis=0)

test_input = test_input / 255.0

y_pre = model_resnet50.predict(test_input)

y_classes = [np.argmax(y_pre)][0]
# print(y_pre)
print(y_classes)
print(categori_classname[y_classes])
plt.imshow(test_img_resized)
plt.show()

In [ ]:
model_resnet50.save('model_resnet50_animal_90_ver2.h5')

# EfficientNetB3

In [ ]:
%%time
print(color.BOLD)
EFF_BATCH_SIZE = 15
EFF_IMAGE_SIZE = (224, 224)

EfficentNetB3_generator = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
    # there could be image augmentation here
    #rescale = 1./255,
    #rotation_range = 20,
    #width_shift_range = 0.1,
    #height_shift_range = 0.1,
    #shear_range = 0.1,
    #zoom_range = 0.1,
    #fill_mode = 'nearest',
    #horizontal_flip = True,
    #vertical_flip = True,
)

# Split the data into three categories.
train_eff = EfficentNetB3_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='imgpath',
    y_col='labels',
    target_size=EFF_IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=EFF_BATCH_SIZE,
    shuffle=True,
    seed=42,
)

val_eff = EfficentNetB3_generator.flow_from_dataframe(
    dataframe=valid_df,
    x_col='imgpath',
    y_col='labels',
    target_size=EFF_IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=EFF_BATCH_SIZE,
    shuffle=False
)

test_eff = EfficentNetB3_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='imgpath',
    y_col='labels',
    target_size=EFF_IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=EFF_BATCH_SIZE,
    shuffle=False
)

In [ ]:
print(color.BOLD)
# Load the pretained model
eff_model = tf.keras.applications.EfficientNetB3(
    input_shape=(224, 224, 3),
    include_top=False, # we don`t need a pre-trained top layer (output layer)
    weights='imagenet',
    pooling='max'
)

for i ,layer in enumerate(eff_model.layers):
    eff_model.layers[i].trainable = False

In [ ]:
num_classes = len(set(train_eff.classes))
print(num_classes)

In [ ]:

    # Data Augmentation Step
model_efficientnet = tf.keras.Sequential([
        eff_model,
        layers.Flatten(),
        layers.Dense(256,activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.45),
        layers.Dense(num_classes,activation = 'softmax')
])

#model_efficentnet = tf.keras.Sequential()

#inputs = layers.Input(shape = (224,224,3), name='inputLayer')
#x = model_efficentnet(inputs)
#pretrain_out = eff_model(x, training = False)
#x = layers.Dense(256)(pretrain_out)
#x = layers.Activation(activation="relu")(x) 
#x = BatchNormalization()(x)
#x = layers.Dropout(0.45)(x)
#x = layers.Dense(num_classes)(x)
#outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x) 
# mixed_precision need separated Dense and Activation layers
#model = Model(inputs=inputs, outputs=outputs)



model_efficientnet.compile(
    optimizer=Adam(0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']

)
# model.load_weights('./checkpoints/my_checkpoint')

print(color.BOLD)
print(model_efficientnet.summary())

In [ ]:
history = model_efficientnet.fit(
    train_eff,
    steps_per_epoch=len(train_eff),
    validation_data=val_eff,
    validation_steps=len(val_eff),
    epochs=15,
    callbacks=[
        EarlyStopping(monitor = "val_loss",patience = 3,restore_best_weights = True), 
        # if val loss decreases for 10 epochs in a row, stop training,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min') 
    ]
)

In [ ]:
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
Epochs = [i + 1 for i in range(len(tr_acc))]
loss_label = 'Best epoch: {}'.format(str(index_loss + 1))
acc_label = 'Best epoch: {}'.format(str(index_acc + 1))

plt.figure(figsize = (20,8))
plt.style.use('fivethirtyeight')

plt.subplot(1,2,1)
plt.plot(Epochs,tr_loss,'r',label = 'Training Loss')
plt.plot(Epochs,val_loss,'c',label = 'Validation Loss')
plt.scatter(index_loss + 1,val_lowest,s = 150,c = 'blue',label =  loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(Epochs,tr_acc,'r',label = 'Training Accuracy')
plt.plot(Epochs,val_acc,'c',label = 'Validation Accuracy')
plt.scatter(index_acc + 1,acc_highest, s =150,c = 'blue',label = acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

In [ ]:
results_1 = model_efficientnet.evaluate(test_eff,verbose = 0)

print('Test Loss: {:.2f}%'.format(results_1[0]*100))
print('Test Accuracy: {:.2f}%'.format(results_1[1]*100))

In [ ]:
# model.save('model1_animal_90.h5')

In [ ]:
eff_model.trainable = True
for layer in eff_model.layers:
    if isinstance(layer, layers.BatchNormalization): # set BatchNorm layers as not trainable
        layer.trainable = False
        
# let`s see first 10 layers
for l in eff_model.layers[:10]:
    print(l.name, l.trainable)

model_efficientnet.compile(
    optimizer = Adam(0.00001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model_efficientnet.summary()

In [ ]:
history = model_efficientnet.fit(
    train_eff,
    steps_per_epoch = len(train_eff),
    validation_data = val_eff,
    validation_steps = len(val_eff),
    epochs = 12,
    callbacks = [
        EarlyStopping(monitor = 'val_loss',patience = 3,restore_best_weights = True),
        ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 2, mode = 'min')
    ]
)

# model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
colors = ['#EF7D71','#41ABD7','#36609A','#FFCF30','#194350']

fig,ax = plt.subplots(2,1,figsize = (8,8), dpi= 100)
fig.patch.set_facecolor('#f5f6f6')

axes = ax.ravel()

for ax in axes:
    ax.set_facecolor('#f5f6f6')
    for loc in ['right','top']:
        ax.spines[loc].set_visible(False)
        
hist1 = history.history
Epochs = range(len(hist1['loss']))

sns.lineplot(x = Epochs, y = hist1['val_loss'], ax = axes[0],linewidth = 4,color = colors[3])
sns.lineplot(x = Epochs, y = hist1['loss'],ax = axes[0],linewidth = 4,color = colors[4])

axes[0].text(Epochs[-1] + 0.25,hist1['val_loss'][-1],'Validation Loss',
             {'fontfamily':'serif','size': 12, 'weight': 'bold','color': colors[3]})
axes[0].text(Epochs[-1] + 0.25,hist1['loss'][-1],'Training Loss',
             {'fontfamily':'serif', 'size': 12,'weight': 'bold', 'color': colors[4]})

sns.lineplot(x = Epochs, y =hist1['val_accuracy'],ax = axes[1],linewidth = 4,color = colors[3])
sns.lineplot(x = Epochs, y = hist1['accuracy'],ax = axes[1], linewidth = 4,color = colors[4])

axes[1].text(Epochs[-1] + 0.25, hist1['val_accuracy'][-1],'Validation Accuracy',
             {'fontfamily': 'serif', 'size':12,'weight': 'bold', 'color' : colors[3]})
axes[1].text(Epochs[-1] + 0.25, hist1['accuracy'][-1],'Training Accuracy',
             {'fontfamily': 'serif','size':12 , 'weight': 'bold', 'color' : colors[4]})

plt.tight_layout(h_pad = 5)
fig.show()

In [ ]:
results_2 = model_efficientnet.evaluate(test_eff,verbose = 0)

print('Test Loss: {:.2f}%'.format(results_2[0]*100))
print('Test Accuracy: {:.2f}%'.format(results_2[1]*100))

In [ ]:
# model.save('model2_animal_90.h5')

In [ ]:
y_true = test_eff.classes
y_pre = np.argmax(model_efficientnet.predict(test_eff),axis=1)
f1 = f1_score(y_true,y_pre,average='macro')
print('F1 Score: {}'.format(f1))
print(classification_report(y_true,y_pre,target_names = test_eff.class_indices.keys()))

In [ ]:
# Kiểm tra độ dài của mỗi mảng
classes = dict(zip(test_eff.class_indices.values(),test_eff.class_indices.keys()))
print("Image Index length:", len(list(range(len(test_eff)))))
print("Test Labels length:", len(test_eff.labels))
print("Test Classes length:", len([classes[i] for i in test_eff.labels]))
print("Prediction Labels length:", len(y_pre))
print("Prediction Classes length:", len([classes[i] for i in y_pre]))
print("Path length:", len(test_eff.filenames))
print("Prediction Probability length:", len([x for x in np.asarray(tf.reduce_max(model_efficientnet.predict(test_eff), axis=1))]))

In [ ]:
Predictions = pd.DataFrame({'Image Index': list(range(len(test_eff.labels))),
                           'Test Labels': test_eff.labels,
                           'Test Classes': [classes[i] for i in test_eff.labels],
                            'Prediction Labels': y_pre,
                            'Prediction Classes': [classes[i] for i in y_pre],
                            'Path': test_eff.filenames,
                            'Prediction Probability': [x for x in np.asarray(tf.reduce_max(model_efficientnet.predict(test_eff),axis = 1))]
                           })

Predictions.head(20)

In [ ]:
# classes = dict(zip(test_images.class_indices.keys()))
# cate = test_images.class_indices.keys()
# print(cate)

In [ ]:
plt.figure(figsize=(30,30))
for i,row in Predictions[Predictions['Test Labels'] != Predictions['Prediction Labels']].sort_values('Prediction Probability').tail(20).reset_index().iterrows():
    plt.subplot(5,4,i+1)
    image_path = row['Path']
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(f'TRUE: {row["Test Classes"]} | PRED: {row["Prediction Classes"]}', fontsize=8)
    plt.axis('off')
    
plt.show()

In [ ]:
preds = model_efficientnet.predict_generator(test_eff)
y_pre = np.argmax(preds, axis=1)
g_dict = test_eff.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_eff.classes, y_pre)

plt.figure(figsize= (30, 30))
plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation= 45)
plt.yticks(tick_marks, classes)


thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')

plt.tight_layout()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.show()

In [ ]:
model.compile(
    optimizer = Adam(0.00005),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    train_images,
    steps_per_epoch = len(train_images),
    validation_data = val_images,
    validation_steps = len(val_images),
    epochs = 20,
    callbacks = [
        ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 2, mode = 'min')
    ]
)

In [ ]:
results_3 = model.evaluate(test_images,verbose = 0)

print('Test Loss: {:.2f}%'.format(results_3[0]*100))
print('Test Accuracy: {:.2f}%'.format(results_3[1]*100))

In [ ]:
# model.save('model3_animal_90.h5')

In [ ]:
categori_classname = ['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle',
                      'bison', 'boar', 'butterfly', 'cat', 'caterpillar',
                      'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab',
                      'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly',
                      'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox',
                      'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper',
                      'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill',
                      'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo',
                      'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster',
                      'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan',
                      'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes',
                      'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat',
                      'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark',
                      'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish',
                      'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf',
                      'wombat', 'woodpecker', 'zebra']

# Load test image
# model = keras.models.load_model('/kaggle/input/model2_animal_90/keras/model2_animal_90/1/model2_animal_90.h5')
test_img = cv2.imread('/kaggle/input/animal-image-dataset-90-different-animals/animals/animals/bat/330e4a8053.jpg')
# Resize test image to match model input size (224x224)
test_img_resized = cv2.resize(test_img, (224, 224))
# Convert test image to numpy array
#test_input = np.array(test_img_resized)
test_input = keras.applications.efficientnet.preprocess_input(test_img_resized)
# Expand dimensions to match model input shape (add batch dimension)
test_input = np.expand_dims(test_input, axis=0)
# Predict
#test_input = test_input / 255.0
y_pre = model_efficientnet.predict(test_input)
# Get predicted class
#y_classes = [np.argmax(y_pre)][0]
y_classes = np.argmax(y_pre)
print(y_pre)
print(y_classes)
# Print predicted class
print(categori_classname[y_classes])
# Display the test image
plt.imshow(test_img_resized)
plt.show()